<a href="https://colab.research.google.com/github/albert-yue/gcn-explanability/blob/master/gcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd gdrive/My\ Drive/gcn_explainability

/content/gdrive/My Drive/gcn_explainability


In [9]:
! git pull

Already up to date.


In [13]:
%ls

data/  gcn.ipynb  logs.pt  notebooks/  README.md  src/


In [3]:
from src.models.gcn import GCN
import torch
import numpy as np
import scipy.sparse as sparse

logs = (200, 0.5, 41664, 23, 'logs.pt')
ng20 = (200, 0.5, 80035, 20, 'gcn_20ng.pt')

hidden_size,dropout,num_vertices,num_labels,path = ng20

model = GCN(num_vertices, hidden_size, num_labels, dropout=dropout)
model.load_state_dict(torch.load(path))
model.eval()

GCN(
  (dropout): Dropout(p=0.5, inplace=False)
  (act_func): ReLU()
  (softmax): LogSoftmax()
  (layer1): GraphConv(input_size=80035, output_size=200)
  (layer2): GraphConv(input_size=200, output_size=20)
)

In [0]:
from src.data import Corpus, get_data, get_labels, get_vocabulary
from src.utils import load_sparse_tensor

vocabulary = get_vocabulary('data/20ng-vocabulary.txt')
labels = get_labels('data/20ng-labels.txt')
test_corpus = get_data('data/test-20news-clean.txt', labels)
test_adj_matrix = load_sparse_tensor('data/20ng_test_adj_matrix_norm.pt')


In [0]:
import torch.nn as nn
from torch.autograd import Variable

loss_fn = nn.CrossEntropyLoss()
#input_value = Variable(test_adj_matrix, requires_grad=True)
model.zero_grad()
output = model(test_adj_matrix)
loss = loss_fn(output[-7532:,:], test_corpus.labels())
loss.retain_grad()
loss.backward()



In [12]:
weight_matrices = []
gradients = []
for n, p in model.named_parameters():
  print(n)
  weight_matrices.append(p.detach().numpy())
  gradients.append(p.grad.numpy())
  print(p.grad.shape)

#weight_matrices[1] = weight_matrices[1].T
#gradients[1] = gradients[1].T

layer1.weight
torch.Size([80035, 200])
layer2.weight
torch.Size([200, 20])


In [0]:
adj = test_adj_matrix.coalesce()
V = sparse.csr_matrix((adj.values(), adj.indices()), shape=list(adj.size()))
F_1 = V.dot(weight_matrices[0])
F_2 = V.dot(np.matmul(F_1, weight_matrices[1]))
F = [F_1, F_2]
grad_cams = []
for l in range(2):
  grad_cam = []
  grad_cams.append(grad_cam)
  alphas = np.mean(gradients[l], axis = 0)
  for n in range(num_vertices):
    grad_cam.append(np.maximum(0, np.dot(alphas, F[l][n, :])))

grad_cam_avg = np.mean(grad_cams, axis=0)

In [25]:
print(grad_cam_avg)

[0.14636354 0.06648389 0.00815528 ... 0.02801099 0.03144196 0.01767992]


In [35]:
!git push

fatal: could not read Username for 'https://github.com': No such device or address
